# AADT Confidence Interval, I-605 District 7

## Helpful Links

---

### FHWA Links
* Guidelines for Obtaining AADT Estimates from Non-Traditional Sources:
    * https://www.fhwa.dot.gov/policyinformation/travel_monitoring/pubs/aadtnt/Guidelines_for_AADT_Estimates_Final.pdf

---
  
### AADT Analysis Locations
* 10 locations were used in the analysis
* Locations were determined based on the location on installed & recording Traffic Operations cameras
    * for additional information contact Zhenyu Zhu with Traffic Operations

### Traffic Census Data
* https://dot.ca.gov/programs/traffic-operations/census/traffic-volumes
* Back AADT, Peak Month, and Peak Hour usually represents traffic South or West of the count location.  
* Ahead AADT, Peak Month, and Peak Hour usually represents traffic North or East of the count location. Listing of routes with their designated  

* Because the Back & Ahead counts are included at each location in the Traffic Census Data, (e.g., "IRWINDALE, ARROW HIGHWAY") only one [OBJECTID*] per location was pulled; for this analysis the North Bound Nodes were used for the analysis. 
    * for more information see the diagram: https://traffic.onramp.dot.ca.gov/downloads/traffic/files/performance/census/Back_and_Ahead_Leg_Traffic_Count_Diagram.pdf

### StreetLight Analysis Data
* Analysis Type == Network Performance
* Segment Metrics
* 2022 was used to match currently available Traffic Census Data (as of 8/27/2025)
* pulled a variety of Day Types, but plan to just look at """All Day Types"""
* pulled a variety of Day Parts, but plan to just look at """All Day Parts"""

---


## Identify the corridor

In [1]:
# pull in the coordinates from the utils docs
#from osow_frp_o_d_utils_v3 import origin_intersections, destination_intersections
import shs_ct_tc_locations_utils as tc_locs

In [2]:
# Available corridors
    # "interstate_605_d7_tc_aadt_locations"
    # "sr_99_d3_tc_aadt_locations"

### Update the corridor name

In [3]:
# Identify the corridor to be analyzed
CORRIDOR_VAR_NAME = "interstate_605_d7_tc_aadt_locations"
#CORRIDOR_VAR_NAME = "sr_99_d3_tc_aadt_locations"

In [4]:
# Resolve the object from the module by name
try:
    aadt_locations = getattr(tc_locs, CORRIDOR_VAR_NAME)
except AttributeError:
    raise KeyError(
        f"'{CORRIDOR_VAR_NAME}' not found in shs_ct_tc_locations_utils. "
        "Double-check the variable name."
    )

## import packages

In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import t as student_t  # if SciPy is not available, use a small lookup table
import csv
import re

from pathlib import Path

## Step 0, Pull in the Data

In [6]:
# ------------------------------------------------------
# 0) Pull in the Data
# ------------------------------------------------------

# This function will pull in the data and clean the column headers in a way that will make them easier to work with
def getdata_and_cleanheaders(path):
    # Read the CSV file
    df = pd.read_csv(path)

    # Clean column headers: remove spaces, convert to lowercase, and strip trailing asterisks
    cleaned_columns = []
    for column in df.columns:
        cleaned_column = column.replace(" ", "").lower().rstrip("*")
        cleaned_columns.append(cleaned_column)

    df.columns = cleaned_columns
    return df

### import option 0.1: Identify the Google Cloud Storage path

In [7]:
# # Identify the GCS path to the data
# gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/compare_traffic_counts/0_2022/"

In [8]:
# pull in the data & create dataframes
#df_tc = getdata_and_cleanheaders(f"{gcs_path}caltrans_traffic_census_2022.csv")  # Traffic Census

In [9]:
# # Identify the StreetLight Analysis to be used in the AADT comparison
# df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_605_d7_all_vehicles_np_2022.csv")  # StreetLight

### import option 0.2: Identify the local data path

In [10]:
# Base data folder: aadt_confidence_interval/aadt_data/2022
LOCAL_DATA_DIR = Path.cwd() / "aadt_data" / "2022"
if not LOCAL_DATA_DIR.exists():
    raise FileNotFoundError(f"Data folder not found: {LOCAL_DATA_DIR.resolve()}")

In [11]:
# Traffic Census (traditional) — local CSV
df_tc = getdata_and_cleanheaders(LOCAL_DATA_DIR / "caltrans_traffic_census_2022.csv")

In [12]:
# Available datasets
    # streetlight_605_d7_all_vehicles_np_2022.csv
    # streetlight_99_d3_all_vehicles_2022_np.csv

In [13]:
# StreetLight (non-traditional) — local CSV
df_stl = getdata_and_cleanheaders(LOCAL_DATA_DIR / "streetlight_605_d7_all_vehicles_np_2022.csv")
#df_stl = getdata_and_cleanheaders(LOCAL_DATA_DIR / "streetlight_99_d3_all_vehicles_2022_np.csv")

### Export to a CSV for viewing/validation

In [14]:
# export traditional dataset (Traffic Census
df_tc.to_csv("df_tc.csv", index=False)

# export non-traditional dataset (StreetLight)
df_stl.to_csv("df_stl.csv", index=False)

## Step 1, (moved to step 6)

## Step 2, Build a per-location summary of Traffic Census locations

* **Goal:** For each location, pick the relevant **Traffic Census node** (objectid's), compute a representative **traditional AADT**, and emit a tidy per-location record.
* **OBJECTID selection policy**: Prefer **ODD** objectid's, if non available, use **all** available.
* **Outputs (one row per location):**
    * location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids, traditional_ahead_mean, traditional_behind_mean, traditional_aadt
* **Why this matters:** We now have a clean, comparable **traditional AADT** baseline per location to contrast with the non-traditional estimates in later steps

In [15]:
# ------------------------------------------------------------
# 2) Build a per-location summary of Traffic Census locations
# ------------------------------------------------------------


def traditional_aadt_by_location(aadt_locations, df_tc, as_df=True, use_parity=False):
    """
    Build a per-location summary of *traditional* (Traffic Census) AADT.

    Policy:
      • If multiple objectids exist for a location, prefer those whose numeric value is ODD.
        If no odd ids exist, fall back to all ids found.
      • Default behavior (use_parity=False): for each kept objectid, compute (ahead_aadt + back_aadt)/2,
        then average across kept objectids.
        
    Output columns:
      location, daytype, objectids, n_objectids, n_found_in_tc, missing_objectids,
      traditional_ahead_mean, traditional_behind_mean, traditional_aadt
    """
    # Requires: import pandas as pd; import numpy as np

    def _ensure_list(x):
        if x is None: return []
        if isinstance(x, (list, tuple, set)): return list(x)
        return [x]

    def _gather_objectids(node_dict):
        ids = []
        if not isinstance(node_dict, dict): return ids
        if "objectid"  in node_dict: ids.extend(_ensure_list(node_dict["objectid"]))
        if "objectids" in node_dict: ids.extend(_ensure_list(node_dict["objectids"]))
        return [str(i).strip() for i in ids if i is not None and str(i).strip() != ""]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _keep_odd_objectids(ids):
        odds = [i for i in ids if i.isdigit() and (int(i) % 2 == 1)]
        return odds if odds else ids

    def _normalize_one_location(name, loc, include_oid_in_name=True):
        nodes = (loc.get("nodes") if isinstance(loc, dict) else None) or {}
        all_ids=[]
        for _, node in nodes.items():
            all_ids.extend(_gather_objectids(node))
        if not all_ids and isinstance(loc, dict) and "objectid" in loc:
            all_ids = [str(loc["objectid"])]

        all_ids = _dedup([i for i in all_ids if i])
        kept_ids = _keep_odd_objectids(all_ids)

        name_out = name
        if include_oid_in_name and kept_ids:
            name_out = f"{name} [{','.join(kept_ids)}]"

        return {
            "name": name_out,
            "daytype": (loc.get("daytype") if isinstance(loc, dict) else None) or "0: All Days (M-Su)",
            "objectids": kept_ids,
        }

    def _normalize_input(aadt_locs):
        if isinstance(aadt_locs, pd.DataFrame) and {"name","daytype","objectids"}.issubset(aadt_locs.columns):
            recs = aadt_locs.to_dict(orient="records")
            for r in recs:
                r["objectids"] = _keep_odd_objectids(_ensure_list(r.get("objectids")))
            return recs
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","objectids"}.issubset(aadt_locs[0].keys()):
            recs = []
            for r in aadt_locs:
                r = dict(r)
                r["objectids"] = _keep_odd_objectids(_ensure_list(r.get("objectids")))
                recs.append(r)
            return recs

        recs = []
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
            return recs

        if isinstance(aadt_locs, list):
            for item in aadt_locs:
                if not isinstance(item, dict):
                    continue
                if "nodes" in item:
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif "objectid" in item:
                    oid = str(item.get("objectid")).strip()
                    nm  = item.get("location_description") or item.get("name") or "UNKNOWN"
                    kept = _keep_odd_objectids([oid])
                    recs.append({
                        "name": f"{nm} [{','.join(kept)}]" if kept else nm,
                        "daytype": item.get("daytype", "0: All Days (M-Su)"),
                        "objectids": kept,
                    })
                else:
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    def _traditional_aadt_for_ids(df_tc_in, obj_ids):
        """
        Default (use_parity=False): per-oid average of (ahead_aadt, back_aadt), then mean across oids.
        If use_parity=True: even->back_aadt, odd->ahead_aadt.
        """
        obj_ids = [str(x).strip() for x in (obj_ids or []) if str(x).strip()]
        if not obj_ids:
            return np.nan, np.nan, np.nan, 0

        sub = df_tc_in[df_tc_in["objectid"].astype(str).str.strip().isin(obj_ids)].copy()
        if sub.empty:
            return np.nan, np.nan, np.nan, 0

        if use_parity:
            vals = []
            for oid in obj_ids:
                row = sub[sub["objectid"].astype(str).str.strip() == oid]
                if row.empty:
                    continue
                v = row.iloc[0]["back_aadt"] if (oid.isdigit() and int(oid) % 2 == 0) else row.iloc[0]["ahead_aadt"]
                vals.append(pd.to_numeric(v, errors="coerce"))
            vals = pd.Series(vals, dtype="float64").dropna()
            if vals.empty: return np.nan, np.nan, np.nan, 0
            overall = float(vals.mean())
            return overall, np.nan, np.nan, int(vals.shape[0])

        # --- average ahead/back per objectid, then average across objectids ---
        sub["ahead_aadt"] = pd.to_numeric(sub.get("ahead_aadt"), errors="coerce")
        sub["back_aadt"]  = pd.to_numeric(sub.get("back_aadt"),  errors="coerce")

        # per-oid average: mean of available sides (ignore NaN)
        per_oid_avg = sub[["ahead_aadt","back_aadt"]].mean(axis=1, skipna=True)
        per_oid_avg = per_oid_avg.dropna()

        if per_oid_avg.empty:
            return np.nan, np.nan, np.nan, 0

        overall = float(per_oid_avg.mean())

        # side means (for reporting only)
        ahead_vals = sub["ahead_aadt"].dropna()
        back_vals  = sub["back_aadt"].dropna()
        mean_ahead = float(ahead_vals.mean()) if not ahead_vals.empty else np.nan
        mean_back  = float(back_vals.mean())  if not back_vals.empty  else np.nan
        count_used = int(per_oid_avg.shape[0])

        return overall, mean_ahead, mean_back, count_used

    # ---- main ----
    norm = _normalize_input(aadt_locations)
    tc_ids_all = set(df_tc["objectid"].astype(str).str.strip().unique())

    rows = []
    for loc in norm:
        obj_ids = [str(x).strip() for x in (loc.get("objectids") or []) if str(x).strip()]
        overall, mean_ahead, mean_back, n_found = _traditional_aadt_for_ids(df_tc, obj_ids)
        missing = [x for x in obj_ids if x not in tc_ids_all]

        rows.append({
            "location": loc.get("name"),
            "daytype":  loc.get("daytype"),
            "objectids": "|".join(obj_ids),
            "n_objectids": len(obj_ids),
            "n_found_in_tc": int(n_found),
            "missing_objectids": "|".join(missing) if missing else "",
            "traditional_ahead_mean": mean_ahead,
            "traditional_behind_mean": mean_back,
            "traditional_aadt": overall,
        })

    return pd.DataFrame(rows) if as_df else rows

In [16]:
# run step 2 - traditional aadt counts
trad_df = traditional_aadt_by_location(aadt_locations, df_tc, as_df=True)

In [17]:
# Export Step 1 as a CSV to take a look
trad_df.to_csv("step_2_traditional_aadt_by_location_605_d7.csv", index=False)

## Step 3, Non-Traditional AADT Summarizer

* **Goal:** For each locaiton, use its **StreetLight segment lists** ahead_zones/behind_zones (and also keep the "all" lists before de-dup for optional counts
* **How it works:**
    1. **Normalize locations** to get clean, de-duplicated ahead_zones/behind_zones (and also keep the "all" lists before de-dup for optional counts). 
    2. **Filter StreetLight rows** by daytype, daypart, and optional modeoftravel
    3. **Collapse StreetLight by segment:** groupby(zonename) to get a per-segment mean volume and a row count backing that mean
    4. For each location/side:
        * Pick which list to use for aggregation (**unique** segments for **all** before de-dup)
        * Pull the per-segment means for segments that are present; record any **missing** segments
        * **Aggregate within a side** using *agg* (**sum** by default; or *mean* if chosen).
    5. **Combine directions**: average the two sides when both exist otherwise take the side that exists. 
* **Outputs (per location)**: metadata (filters used), the zone list, **per-side values, overall non-traditional AADT, backing row counts,** and **segment presence/missing** diagnostics. 
* **Why this matters:** Produces a consistent, auditable **Non-Traditional baseline** to compare with the traditional AADT from Step 2.


In [18]:
# ------------------------------------------------------
# 3) Non-Traditional AADT Summarizer
# ------------------------------------------------------

def non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True,
    agg="sum",                # "sum" mirrors reviewed pipeline at the per-side level
    segment_count_mode="unique",  # "unique" counts deduped zonenames; "all" counts before dedup
):
    """
    Build a per-location summary of *non-traditional* AADT.

    Output columns (one row per location):
      location, daytype_expected, daytype_used, daypart_used, modeoftravel_used,
      ahead_zones, behind_zones,
      non_trad_ahead_mean, non_trad_behind_mean, non_trad_aadt,
      stl_ahead_rows, stl_behind_rows, missing_ahead_zones, missing_behind_zones,
      listed_ahead_segments, listed_behind_segments, present_ahead_segments, present_behind_segments
    """

    # ---- helpers ----
    def _ensure_list(x):
        if x is None:
            return []
        if isinstance(x, (list, tuple, set)):
            return list(x)
        return [x]

    def _gather_zones(node_dict):
        ahead  = _ensure_list(node_dict.get("zonename_ahead", []))
        behind = _ensure_list(node_dict.get("zonename_behind", []))
        return ahead, behind

    def _gather_objectids(node_dict):
        ids = []
        if not isinstance(node_dict, dict):
            return ids
        if "objectid"  in node_dict: ids.extend(_ensure_list(node_dict["objectid"]))
        if "objectids" in node_dict: ids.extend(_ensure_list(node_dict["objectids"]))
        return [str(i) for i in ids if i is not None and str(i).strip() != ""]

    def _dedup(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out

    def _normalize_one_location(name, loc, include_oid_in_name=True):
        """Collect objectids and zonenames; append [oids] to name for merge alignment."""
        nodes = loc.get("nodes", {}) or {}
        ahead_all, behind_all, all_oids = [], [], []
        for _, node in nodes.items():
            a, b = _gather_zones(node)
            ahead_all.extend([z for z in a if z])
            behind_all.extend([z for z in b if z])
            all_oids.extend(_gather_objectids(node))

        name_out = name
        if include_oid_in_name and all_oids:
            name_out = f"{name} [{','.join(_dedup(all_oids))}]"

        return {
            "name": name_out,
            "daytype": loc.get("daytype", "0: All Days (M-Su)"),
            "ahead_zones_all": ahead_all,
            "behind_zones_all": behind_all,
            "ahead_zones": _dedup(ahead_all),
            "behind_zones": _dedup(behind_all),
        }

    def _normalize_input(aadt_locs):
        # Already normalized DataFrame?
        if isinstance(aadt_locs, pd.DataFrame) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs.columns):
            recs = aadt_locs.to_dict(orient="records")
            for r in recs:
                r.setdefault("ahead_zones_all", r.get("ahead_zones", []))
                r.setdefault("behind_zones_all", r.get("behind_zones", []))
            return recs

        # Already normalized list[dict]?
        if isinstance(aadt_locs, list) and aadt_locs and isinstance(aadt_locs[0], dict) and \
           {"name","daytype","ahead_zones","behind_zones"}.issubset(aadt_locs[0].keys()):
            recs = aadt_locs
            for r in recs:
                r.setdefault("ahead_zones_all", r.get("ahead_zones", []))
                r.setdefault("behind_zones_all", r.get("behind_zones", []))
            return recs

        recs = []
        # Dict keyed by name (nested format)
        if isinstance(aadt_locs, dict):
            for nm, loc in aadt_locs.items():
                recs.append(_normalize_one_location(nm, loc))
            return recs

        # List of locations (mixed formats)
        if isinstance(aadt_locs, list):
            for item in aadt_locs:
                if not isinstance(item, dict):
                    continue
                if "nodes" in item:
                    nm = item.get("location_description") or item.get("name") or "UNKNOWN"
                    recs.append(_normalize_one_location(nm, item))
                elif "objectid" in item:
                    # flat row (objectid + zonename_0..3)
                    oid = str(item.get("objectid"))
                    nm  = item.get("location_description") or item.get("name") or "UNKNOWN"
                    day = item.get("daytype", "0: All Days (M-Su)")

                    ahead_all, behind_all = [], []
                    for k, v in item.items():
                        if not (isinstance(k, str) and k.startswith("zonename_")):
                            continue
                        try:
                            idx = int(k.split("_")[1])
                        except Exception:
                            idx = None
                        # even 0/2 -> ahead; odd 1/3 -> behind
                        if idx is not None and idx % 2 == 0:
                            ahead_all.append(v)
                        else:
                            behind_all.append(v)

                    recs.append({
                        "name": f"{nm} [{oid}]",
                        "daytype": day,
                        "ahead_zones_all": ahead_all,
                        "behind_zones_all": behind_all,
                        "ahead_zones": _dedup([z for z in ahead_all if z]),
                        "behind_zones": _dedup([z for z in behind_all if z]),
                    })
                else:
                    for nm, loc in item.items():
                        recs.append(_normalize_one_location(nm, loc))
        return recs

    # ---- filter & precompute per-zone means ----
    must_cols = [zonename_col, stl_volume_col, "daytype", "daypart"]
    for c in must_cols:
        if c not in df_stl.columns:
            raise KeyError(f"df_stl is missing required column: {c}")

    filt = (df_stl["daytype"] == daytype_filter) & (df_stl["daypart"] == daypart_filter)
    if modeoftravel_filter and ("modeoftravel" in df_stl.columns):
        filt = filt & (df_stl["modeoftravel"] == modeoftravel_filter)

    stl_filtered = df_stl.loc[filt, [zonename_col, stl_volume_col]].copy()

    # Clean types
    stl_filtered[zonename_col] = stl_filtered[zonename_col].astype(str).str.strip()
    stl_filtered[stl_volume_col] = pd.to_numeric(stl_filtered[stl_volume_col], errors="coerce")

    # Per-zonename averages (handles duplicates safely)
    zone_group = stl_filtered.groupby(zonename_col)[stl_volume_col]
    zone_mean = zone_group.mean()   # pd.Series: index=zonename, value=mean volume
    zone_rows = zone_group.size()   # pd.Series: index=zonename, value=row count backing the mean
    present_zones = set(zone_mean.index)

    def _zone_stats(zones_list, agg_local="sum"):
        """
        Return:
          aggregated_value,
          backing_row_count_sum,
          missing_list,
          present_segment_count,
          listed_segment_count
        """
        zones_all = [z for z in _ensure_list(zones_list) if z and str(z).strip() != ""]
        zones_all = [str(z).strip() for z in zones_all]
        zones_for_agg = _dedup(zones_all) if segment_count_mode == "unique" else zones_all

        if not zones_for_agg:
            return np.nan, 0, [], 0, 0

        present = [z for z in zones_for_agg if z in present_zones]
        missing = [z for z in zones_for_agg if z not in present_zones]

        vals = zone_mean.reindex(present).dropna()
        if agg_local == "sum":
            val = float(vals.sum()) if len(vals) else np.nan
        else:  # "mean" across segments within a side
            val = float(vals.mean()) if len(vals) else np.nan

        n_rows = int(zone_rows.reindex(present).fillna(0).sum())
        present_seg_ct = len(present)
        listed_seg_ct = len(zones_for_agg)
        return val, n_rows, missing, present_seg_ct, listed_seg_ct

    def _combine_dirs(a, b):
        """Average the two directions if both exist; otherwise use the one that exists."""
        a = np.nan if a is None else a
        b = np.nan if b is None else b
        if pd.notna(a) and pd.notna(b):
            return (float(a) + float(b)) / 2.0
        if pd.notna(a):
            return float(a)
        if pd.notna(b):
            return float(b)
        return np.nan

    # ---- build rows ----
    norm = _normalize_input(aadt_locations)
    rows = []
    for loc in norm:
        ahead_all  = loc.get("ahead_zones_all", [])
        behind_all = loc.get("behind_zones_all", [])
        ahead_ded  = loc.get("ahead_zones", [])
        behind_ded = loc.get("behind_zones", [])

        # choose which list drives counts/aggregation
        ahead_for_counts  = ahead_ded if segment_count_mode == "unique" else ahead_all
        behind_for_counts = behind_ded if segment_count_mode == "unique" else behind_all

        val_ahead, ahead_n, miss_a, present_ahead_ct, listed_ahead_ct = _zone_stats(ahead_for_counts, agg_local=agg)
        val_behind, behind_n, miss_b, present_behind_ct, listed_behind_ct = _zone_stats(behind_for_counts, agg_local=agg)

        # Average across directions (NOT mean of means twice; just combine the two sides)
        overall = _combine_dirs(val_ahead, val_behind)

        rows.append({
            "location": loc.get("name"),
            "daytype_expected": loc.get("daytype"),
            "daytype_used": daytype_filter,
            "daypart_used": daypart_filter,
            "modeoftravel_used": modeoftravel_filter if modeoftravel_filter else "",

            # Keep deduped zone strings for readability
            "ahead_zones": "|".join(ahead_ded),
            "behind_zones": "|".join(behind_ded),

            # Values
            "non_trad_ahead_mean": val_ahead,
            "non_trad_behind_mean": val_behind,
            "non_trad_aadt": overall,

            # Backing row counts in df_stl
            "stl_ahead_rows": ahead_n,
            "stl_behind_rows": behind_n,

            # Segment counts
            "listed_ahead_segments": listed_ahead_ct,
            "listed_behind_segments": listed_behind_ct,
            "present_ahead_segments": present_ahead_ct,
            "present_behind_segments": present_behind_ct,

            # Missing lists
            "missing_ahead_zones": "|".join(miss_a) if miss_a else "",
            "missing_behind_zones": "|".join(miss_b) if miss_b else "",
        })

    return pd.DataFrame(rows) if as_df else rows

In [19]:
# this will run the "non_traditional_aadt_by_location" function if  you have the raw nested structure:
stl_df = non_traditional_aadt_by_location(
    aadt_locations,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter="All Vehicles - StL All Vehicles Volume",  # or None
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    as_df=True
)

In [20]:
# Export step 3 to a CSV
stl_df.to_csv("step_3_non_traditional_aadt_by_location_605_d7.csv", index=False)

### Step 4, Build the per-location comparison DataFrame
* **Joining the base location**, not the raw location

-----------------------

* Build inputs with traditional_aadt_by_location(...) and non_traditional_aadt_by_location(...).
* Normalize names to a **base location** by stripping trailing [...] objectids.
* **Merge on** location_base (merge_how="inner" by default; pass "outer" to keep one-sided locations).
* Unify suffixed columns so downstream aggregation sees a consistent schema.
* Collapse duplicates by location_base:
    * Traditional metrics: **mean** across duplicate rows.
    * StreetLight per-direction metrics: **sum** across duplicate rows.
    * Strings (zones, missing): **stable unique union.
    * IDs: **extract digits** and **stable-unique join** with "|"; recompute n_objectids.
* Recompute StreetLight overall: non_trad_aadt = avg(non_trad_ahead_sum, non_trad_behind_sum) (or the lone side if one is missing).
* Compute tce_percent = 100 * (non_trad_aadt - traditional_aadt) / traditional_aadt.
* Return a tidy, consistent column order, with display location set to the base name.

In [21]:
# ------------------------------------------------------
# 4) Build the per-location comparison DataFrame
# ------------------------------------------------------

def build_aadt_comparison_df(
    aadt_locations,
    df_tc,
    df_stl,
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)",
    merge_how="inner",  # use "outer" to keep one-sided locations
) -> pd.DataFrame:
    import re
    import numpy as np
    import pandas as pd

    # ---- local helpers (self-contained) ----
    def _base_location_local(s: str) -> str:
        if not isinstance(s, str): 
            return ""
        return re.sub(r"\s*\[[^\]]+\]\s*$", "", s).strip()

    def _pick_col_local(df: pd.DataFrame, base: str):
        if base in df.columns:
            return base
        for suf in ("_trad", "_nt", "_x", "_y"):
            c = f"{base}{suf}"
            if c in df.columns:
                return c
        return None

    def _unify_cols(df, bases):
        for base in bases:
            c = _pick_col_local(df, base)
            if c and c != base:
                df[base] = df[c]

    def uniq_join(series):
        seen=set(); out=[]
        for s in series.dropna().astype(str):
            for tok in str(s).split("|"):
                tok = tok.strip()
                if tok and tok not in seen:
                    seen.add(tok); out.append(tok)
        return "|".join(out)

    def join_objectids(series):
        toks=[]
        for s in series.astype(str).fillna(""):
            toks.extend(re.findall(r"\d+", s))  # keep digit runs
        out=[]; seen=set()
        for t in toks:
            if t not in seen:
                seen.add(t); out.append(t)
        return "|".join(out)

    # --- 1) Build the two sides ---
    trad_df = traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_tc=df_tc,
        as_df=True
    )
    nt_df = non_traditional_aadt_by_location(
        aadt_locations=aadt_locations,
        df_stl=df_stl,
        daytype_filter=daytype_filter,
        daypart_filter=daypart_filter,
        modeoftravel_filter=modeoftravel_filter,
        zonename_col=zonename_col,
        stl_volume_col=stl_volume_col,
        as_df=True
    )

    # --- add base names BEFORE merging to avoid bracket mismatches ---
    trad_df["location_base"] = trad_df["location"].apply(_base_location_local)
    nt_df["location_base"]   = nt_df["location"].apply(_base_location_local)

    merged = pd.merge(
        trad_df,
        nt_df,
        how=merge_how,
        on="location_base",
        suffixes=("_trad", "_nt")
    )

    # unify to expected column names so the agg works
    _unify_cols(merged, [
        # IDs / metadata
        "objectids","n_objectids","n_found_in_tc",
        "daytype","daytype_expected","daytype_used","daypart_used","modeoftravel_used",
        # zones & missing
        "ahead_zones","behind_zones","missing_ahead_zones","missing_behind_zones",
        # STL counts
        "stl_ahead_rows","stl_behind_rows",
        "listed_ahead_segments","listed_behind_segments",
        "present_ahead_segments","present_behind_segments",
        # metrics
        "traditional_ahead_mean","traditional_behind_mean","traditional_aadt",
        "non_trad_ahead_mean","non_trad_behind_mean",
    ])

    merged["location"] = merged["location_base"]  # display base name

    # --- 3) Collapse duplicates BY location_base ---
    agg = {
        "traditional_ahead_mean": "mean",
        "traditional_behind_mean": "mean",
        "traditional_aadt": "mean",
        "non_trad_ahead_mean": "sum",
        "non_trad_behind_mean": "sum",
        "stl_ahead_rows": "sum",
        "stl_behind_rows": "sum",
        "listed_ahead_segments": "sum",
        "listed_behind_segments": "sum",
        "present_ahead_segments": "sum",
        "present_behind_segments": "sum",
        "ahead_zones": uniq_join,
        "behind_zones": uniq_join,
        "missing_ahead_zones": uniq_join,
        "missing_behind_zones": uniq_join,
        "objectids": join_objectids,
        "n_objectids": "sum",
        "n_found_in_tc": "sum",
        "daytype": "first",
        "daytype_expected": "first",
        "daytype_used": "first",
        "daypart_used": "first",
        "modeoftravel_used": "first",
    }
    agg = {k:v for k,v in agg.items() if k in merged.columns}

    out = (merged
           .groupby("location_base", as_index=False)
           .agg(agg)
           .rename(columns={"location_base":"location"}))

    # Harden objectids; recompute counts from the pipe-joined string
    if "objectids" in out.columns:
        out["objectids"] = out["objectids"].astype(str)
        out["n_objectids"] = out["objectids"].str.split(r"\|").apply(lambda xs: len([t for t in xs if t]))

    # --- 4) Recompute STL combined AADT ---
    if {"non_trad_ahead_mean","non_trad_behind_mean"}.issubset(out.columns):
        a = pd.to_numeric(out["non_trad_ahead_mean"], errors="coerce")
        b = pd.to_numeric(out["non_trad_behind_mean"], errors="coerce")
        out["non_trad_aadt"] = np.where(
            a.notna() & b.notna(), (a + b) / 2.0,
            np.where(a.notna(), a, b)
        )

    # --- 5) Recompute TCE ---
    if {"traditional_aadt","non_trad_aadt"}.issubset(out.columns):
        T = pd.to_numeric(out["traditional_aadt"], errors="coerce")
        N = pd.to_numeric(out["non_trad_aadt"], errors="coerce")
        out["tce_percent"] = np.where(T.notna() & (T != 0) & N.notna(), 100.0*(N - T)/T, np.nan)

    preferred_cols = [
        "location",
        "objectids","n_objectids","n_found_in_tc",
        "ahead_zones","behind_zones",
        "traditional_ahead_mean","traditional_behind_mean","traditional_aadt",
        "non_trad_ahead_mean","non_trad_behind_mean","non_trad_aadt",
        "tce_percent",
        "daytype","daytype_expected","daytype_used","daypart_used","modeoftravel_used",
        "stl_ahead_rows","stl_behind_rows",
        "listed_ahead_segments","listed_behind_segments",
        "present_ahead_segments","present_behind_segments",
        "missing_ahead_zones","missing_behind_zones",
    ]
    cols = [c for c in preferred_cols if c in out.columns]
    return out[cols].copy()

In [22]:
# 4.1, Build the combined comparison DataFrame
cmp_df = build_aadt_comparison_df(
    aadt_locations=aadt_locations,  # your dict/list structure
    df_tc=df_tc,                                 # Traffic Census dataframe
    df_stl=df_stl,                               # StreetLight dataframe
    daytype_filter="0: All Days (M-Su)",
    daypart_filter="0: All Day (12am-12am)",
    modeoftravel_filter=None,                    # e.g., "0: All Modes" if you need it
    zonename_col="zonename",
    stl_volume_col="averagedailysegmenttraffic(stlvolume)"
)

In [23]:
# 4.2, (Optional) sort by absolute TCE to see big deltas first
cmp_df = cmp_df.sort_values("tce_percent", key=lambda s: s.abs(), ascending=False)

In [24]:
# 4.3, Export to CSV 
cmp_df.to_csv("step_4_comparison_dataframe_605_d7.csv", index=False)

### Step 5 Collapse to one row per location

* **Goal**: Guarantee **one row per physical location** (same place can appear multiple times due to multiple objectids, segment splits, or prior joins).

------------------------

* **How it collapses**:
    * Build a **base location key** by stripping trailing ['   '] (spaces) objectid tags
    * **Traditional AADT metrics** (ahead, behind, overall): take the **mean** across duplicates (they should agree or be very close). 
    * **StreetLight per-direction metrics: sum** across duplicates (roll up the per-side contributions). 
    * **Counts** (rows/segments/present segments): **sum** across duplicates.
    * **String fields** (zones, missing lists): **stable unique union.**
    * **Objectids**: extract **digit tokens only**, stable-unique, join with "|"; recompute n_objectids.
* **Then recompute**:
    * *non_trad_aadt* = average of the **summed** ahead/behind values (or the Ione side if one direction is missing). 
    * *tce_percent* = 100 * (*non_trad_aadt - traditional_aadt*) / *traditional_aadt*.

In [25]:
# ------------------------------------------------------
# 5) Collapse to one row per location
# ------------------------------------------------------


def collapse_to_one_row_per_location(cmp_df: pd.DataFrame) -> pd.DataFrame:
    """   
    Collapse possibly-duplicated comparison rows to exactly one row per physical location.

    Policy:
      - Traditional metrics: mean across dup rows (they should agree).
      - StreetLight per-direction values: SUM across dup rows (intended roll-up).
      - Then compute non_trad_aadt = average(ahead_sum, behind_sum), or lone side if only one exists.
      - Keep IDs as pipe-joined strings (digits only for objectids), recompute counts.
      - Idempotent: re-running produces the same numbers.
      """

    df = cmp_df.copy()

    # 1) Normalize to base location (strip trailing " [ ... ]")
    df["location_clean"] = df["location"].astype(str).str.replace(
        r"\s*\[[^\]]+\]\s*$", "", regex=True
    )

    # 2) Helpers ------------------------------------------------------------
    def uniq_join(series):
        # union of pipe-joined tokens, stable order
        seen, out = set(), []
        for s in series.dropna().astype(str):
            for tok in s.split("|"):
                tok = tok.strip()
                if tok and tok not in seen:
                    seen.add(tok); out.append(tok)
        return "|".join(out)

    def join_objectids(series):
        # extract digit runs only (avoids pulling text like "CA 99"), stable-unique
        toks = []
        for s in series.astype(str).fillna(""):
            toks.extend(re.findall(r"\d+", s))
        out, seen = [], set()
        for t in toks:
            if t not in seen:
                seen.add(t); out.append(t)
        return "|".join(out)

    # 3) Aggregation spec (only for columns that exist) ---------------------
    agg = {
        # Traditional should be consistent; mean is safe
        "traditional_ahead_mean": "mean",
        "traditional_behind_mean": "mean",
        "traditional_aadt": "mean",

        # StreetLight per-direction: SUM across dup rows
        "non_trad_ahead_mean": "sum",
        "non_trad_behind_mean": "sum",

        # Counts: sum
        "stl_ahead_rows": "sum",
        "stl_behind_rows": "sum",
        "listed_ahead_segments": "sum",
        "listed_behind_segments": "sum",
        "present_ahead_segments": "sum",
        "present_behind_segments": "sum",

        # Strings/metadata
        "ahead_zones": uniq_join,
        "behind_zones": uniq_join,
        "missing_ahead_zones": uniq_join,
        "missing_behind_zones": uniq_join,
        "daytype": "first",
        "daytype_expected": "first",
        "daytype_used": "first",
        "daypart_used": "first",
        "modeoftravel_used": "first",

        # IDs
        "objectids": join_objectids,
        "n_objectids": "sum",
        "n_found_in_tc": "sum",
    }
    agg = {k: v for k, v in agg.items() if k in df.columns}

    out = (df
           .groupby("location_clean", as_index=False)
           .agg(agg)
           .rename(columns={"location_clean": "location"}))

    # 4) Harden objectids as strings; recompute counts
    if "objectids" in out.columns:
        out["objectids"] = out["objectids"].astype(str)
        out["n_objectids"] = out["objectids"].str.split(r"\|").apply(
            lambda xs: len([t for t in xs if t])
        )

    # 5) Recompute STL combined AADT as avg of per-direction sums (or lone side)
    if {"non_trad_ahead_mean", "non_trad_behind_mean"}.issubset(out.columns):
        a = pd.to_numeric(out["non_trad_ahead_mean"], errors="coerce")
        b = pd.to_numeric(out["non_trad_behind_mean"], errors="coerce")
        out["non_trad_aadt"] = np.where(
            a.notna() & b.notna(), (a + b) / 2.0,
            np.where(a.notna(), a, b)
        )

    # 6) Recompute TCE (%) using updated non_trad_aadt
    if {"traditional_aadt", "non_trad_aadt"}.issubset(out.columns):
        T = pd.to_numeric(out["traditional_aadt"], errors="coerce")
        N = pd.to_numeric(out["non_trad_aadt"], errors="coerce")
        out["tce_percent"] = np.where(
            T.notna() & (T != 0) & N.notna(), 100.0 * (N - T) / T, np.nan
        )

    return out



In [26]:
# run Step 5, the collapse to one row function
cmp_df = collapse_to_one_row_per_location(cmp_df)

In [27]:
# 3.4) Export to CSV 
cmp_df.to_csv("step_5_collapse_to_one_row_605_d7.csv", index=False)

## Step 6 Confidence Interval over TCE

In [28]:
# ------------------------------------------------------
# 6) Confidence interval over TCE
# ------------------------------------------------------


def _prep_tces(
    detail_df,
    tce_col="tce_percent",
    cap_abs=None,
    winsor_pct=None,
    dedupe_by="location",       # NEW: collapse duplicates by this column if present
    dedupe_method="mean"        # "mean" | "median" | "max" | "min"
):
    """
    Prepare a clean 1-D series of TCE values for CI computation.
    - Coerces to numeric, drops inf/NaN.
    - Optionally collapses multiple rows per location (prevents double-weighting).
    - Optional hard cap and winsorization.
    Returns: (Series[float], dropped_count)
    """
    import numpy as np
    import pandas as pd

    # Coerce once
    tce_num = pd.to_numeric(detail_df[tce_col], errors="coerce").replace([np.inf, -np.inf], np.nan)

    # Optional dedupe by location (or other key)
    dedup_dropped = 0
    if dedupe_by is not None and dedupe_by in detail_df.columns:
        df = pd.DataFrame({dedupe_by: detail_df[dedupe_by], "_tce": tce_num})
        if dedupe_method == "median":
            s = df.groupby(dedupe_by)["_tce"].median()
        elif dedupe_method == "max":
            s = df.groupby(dedupe_by)["_tce"].max()
        elif dedupe_method == "min":
            s = df.groupby(dedupe_by)["_tce"].min()
        else:
            s = df.groupby(dedupe_by)["_tce"].mean()  # default mean
        dedup_dropped = int(len(tce_num) - len(s))
    else:
        s = tce_num

    # Drop NaNs
    s = s.dropna()

    # Optional absolute cap
    dropped = dedup_dropped
    if cap_abs is not None:
        mask = s.abs() <= float(cap_abs)
        dropped += int((~mask).sum())
        s = s[mask]

    # Optional winsorization
    if winsor_pct is not None and 0 < winsor_pct < 0.5 and len(s) > 0:
        lo = float(s.quantile(winsor_pct))
        hi = float(s.quantile(1 - winsor_pct))
        s = s.clip(lower=lo, upper=hi)

    return s.astype(float), dropped


def tce_confidence_interval(
    detail_df,
    confidence=0.95,
    tce_col="tce_percent",
    cap_abs=None,           # e.g., 500 trims extreme %s
    winsor_pct=None,        # e.g., 0.01 winsorizes 1% tails
    dedupe_by="location",   # NEW: collapse dup rows by this key if present
    dedupe_method="mean"
):
    """
    One-sample t CI on TCE (%) vs 0.
    Dedupe-by-location prevents accidental double-weighting if prior merges created duplicates.
    Returns: (mean_tce, ci_lo, ci_hi, tcrit, t_stat)
    """
    from scipy import stats
    import numpy as np

    tces, dropped = _prep_tces(
        detail_df, tce_col=tce_col, cap_abs=cap_abs, winsor_pct=winsor_pct,
        dedupe_by=dedupe_by, dedupe_method=dedupe_method
    )
    n = int(tces.shape[0])
    if n == 0:
        return None, None, None, None, None

    mean_tce = float(tces.mean())

    if n > 1:
        std_tce = float(tces.std(ddof=1))
        se = std_tce / np.sqrt(n) if std_tce > 0 else 0.0
        if se > 0:
            dof = n - 1
            tcrit = float(stats.t.ppf((1 + confidence) / 2.0, dof))
            ci_lo = mean_tce - tcrit * se
            ci_hi = mean_tce + tcrit * se
            t_stat = mean_tce / se
        else:
            tcrit = ci_lo = ci_hi = t_stat = None
    else:
        tcrit = ci_lo = ci_hi = t_stat = None

    return mean_tce, ci_lo, ci_hi, tcrit, t_stat


def tce_confidence_interval_df(
    detail_df,
    confidence=0.95,
    tce_col="tce_percent",
    cap_abs=None,           # drop rows with |tce| > cap_abs
    winsor_pct=None,        # winsorize tails by this fraction
    dedupe_by="location",   # NEW
    dedupe_method="mean"    # NEW
) -> pd.DataFrame:
    """
    Same as tce_confidence_interval, with a one-row DataFrame and diagnostics.
    """
    from scipy import stats
    import numpy as np
    import pandas as pd

    tces, dropped = _prep_tces(
        detail_df, tce_col=tce_col, cap_abs=cap_abs, winsor_pct=winsor_pct,
        dedupe_by=dedupe_by, dedupe_method=dedupe_method
    )
    n = int(tces.shape[0])

    if n == 0:
        return pd.DataFrame([{
            "confidence": confidence,
            "tce_col": tce_col,
            "n": 0,
            "dof": None,
            "mean_tce": None,
            "std_tce": None,
            "se": None,
            "t_critical": None,
            "margin_of_error": None,
            "ci_lower": None,
            "ci_upper": None,
            "t_statistic": None,
            "p_value_two_sided": None,
            "cohens_d": None,
            "count_dropped": int(dropped),
            "cap_abs": cap_abs,
            "winsor_pct": winsor_pct,
            "dedupe_by": dedupe_by,
            "dedupe_method": dedupe_method
        }])

    mean_tce = float(tces.mean())

    if n > 1:
        std_tce = float(tces.std(ddof=1))
        se = std_tce / np.sqrt(n) if std_tce > 0 else 0.0
        dof = n - 1

        if se > 0:
            tcrit = float(stats.t.ppf((1 + confidence) / 2.0, dof))
            moe = tcrit * se
            ci_lo = mean_tce - moe
            ci_hi = mean_tce + moe
            t_stat = mean_tce / se
            p_val = float(2 * (1 - stats.t.cdf(abs(t_stat), dof)))
            cohens_d = mean_tce / std_tce if std_tce > 0 else None
        else:
            tcrit = moe = ci_lo = ci_hi = t_stat = p_val = cohens_d = None
    else:
        std_tce = None
        se = None
        dof = None
        tcrit = moe = ci_lo = ci_hi = t_stat = p_val = cohens_d = None

    return pd.DataFrame([{
        "confidence": confidence,
        "tce_col": tce_col,
        "n": n,
        "dof": dof,
        "mean_tce": mean_tce,
        "std_tce": std_tce,
        "se": se,
        "t_critical": tcrit,
        "margin_of_error": moe,
        "ci_lower": ci_lo,
        "ci_upper": ci_hi,
        "t_statistic": t_stat,
        "p_value_two_sided": p_val,
        "cohens_d": cohens_d,
        "count_dropped": int(dropped),
        "cap_abs": cap_abs,
        "winsor_pct": winsor_pct,
        "dedupe_by": dedupe_by,
        "dedupe_method": dedupe_method
    }])


def _ensure_list(x):
    if x is None: return []
    if isinstance(x, (list, tuple, set)): return list(x)
    return [x]

def _dedup_ordered(seq):
    seen=set(); out=[]
    for s in seq:
        if s and s not in seen:
            seen.add(s); out.append(s)
    return out

def _base_location_only(name: str) -> str:
    """
    Strip trailing bracketed [ids] so we can key by the physical location.
    """
    import re
    if not isinstance(name, str): return ""
    return re.sub(r"\s*\[[^\]]+\]\s*$", "", name).strip()

def _pick_global_parity_ids(all_ids_for_loc):
    """
    Apply parity *globally per base location*:
      - If any odd ids exist → keep only the odds (all of them, or just one—see flag below).
      - Else → keep all even ids.
    Return a *stable-deduped* list of kept ids as strings.
    """
    ids = [str(i).strip() for i in all_ids_for_loc if i is not None and str(i).strip()!=""]
    odds = [i for i in ids if i.isdigit() and (int(i) % 2 == 1)]
    kept = odds if odds else ids
    return _dedup_ordered(kept)

def _choose_single_representative_id(kept_ids):
    """
    Optional policy: if you want EXACTLY ONE OID per base location, pick one deterministically.
    Here: pick the smallest numeric id among kept ids.
    """
    nums = [int(i) for i in kept_ids if i.isdigit()]
    if not nums: 
        return kept_ids[0] if kept_ids else None
    rep = str(min(nums))
    return rep

def normalize_locations_one_row_per_base(
    aadt_locations
):
    """
    PSEUDOCODE:
      1) iterate raw locations (nested dict or list)
      2) collect, per base location:
           - all objectids across every node
           - union of ahead_zones across nodes
           - union of behind_zones across nodes
           - daytype (first non-empty)
      3) apply GLOBAL parity on the *union of ids* for that base location
      4) choose either:
           (A) keep ALL kept ids, OR
           (B) keep ONE representative id (recommended to guarantee one row per base)
      5) emit exactly ONE dict per base location with:
           name="<base> [<id or ids>]", daytype, objectids=[...], ahead_zones=[...], behind_zones=[...]
      This shape is accepted by your Step 2/3 builders.
    """
    import re

    # 1) make an iterable over location items of shape {base_name: loc_dict}
    items = []
    if isinstance(aadt_locations, dict):
        items = list(aadt_locations.items())
    elif isinstance(aadt_locations, list):
        # mixed list — support both flat and nested
        for it in aadt_locations:
            if isinstance(it, dict) and "nodes" in it:
                nm = it.get("location_description") or it.get("name") or "UNKNOWN"
                items.append((nm, it))
            elif isinstance(it, dict) and "objectid" in it:
                # flat row → pretend it's a tiny nested record with one node
                nm  = it.get("location_description") or it.get("name") or "UNKNOWN"
                node = {"objectids": [it.get("objectid")]}
                # map zonename_0..3 to ahead/behind by even/odd
                ahead, behind = [], []
                for k, v in it.items():
                    if not isinstance(k, str) or not k.startswith("zonename_"): 
                        continue
                    try:
                        idx = int(k.split("_")[1])
                    except Exception:
                        idx = None
                    if idx is not None and idx % 2 == 0: ahead.append(v)
                    else:                              behind.append(v)
                items.append((nm, {"daytype": it.get("daytype", "0: All Days (M-Su)"),
                                   "nodes": {"_only": {
                                       "objectids": [it.get("objectid")],
                                       "zonename_ahead": ahead,
                                       "zonename_behind": behind
                                   }}}))

    # 2) accumulate per base location
    buckets = {}  # base_name -> dict(daytype, ids(list), ahead(list), behind(list))
    for base_name, loc in items:
        base = _base_location_only(base_name)
        if base not in buckets:
            buckets[base] = {
                "daytype": (loc.get("daytype") if isinstance(loc, dict) else None) or "0: All Days (M-Su)",
                "ids": [],
                "ahead": [],
                "behind": [],
            }
        nodes = (loc.get("nodes") if isinstance(loc, dict) else None) or {}
        for _, node in nodes.items():
            ids = _ensure_list(node.get("objectids", node.get("objectid")))
            ahead = _ensure_list(node.get("zonename_ahead", []))
            behind= _ensure_list(node.get("zonename_behind", []))
            buckets[base]["ids"].extend([str(i).strip() for i in ids if i is not None and str(i).strip()!=""])
            buckets[base]["ahead"].extend([z for z in ahead if z])
            buckets[base]["behind"].extend([z for z in behind if z])

    # 3) parity + 4) pick one representative id (recommended)
    rows = []
    for base, rec in buckets.items():
        kept_ids = _pick_global_parity_ids(rec["ids"])

        # === choose ONE representative id to guarantee one row per location ===
        rep = _choose_single_representative_id(kept_ids)
        if rep is None:
            # if we somehow had no usable ids, skip this location
            continue

        name = f"{base} [{rep}]"  # single-oid label keeps things aligned in downstream merges
        rows.append({
            "name": name,
            "daytype": rec["daytype"],
            "objectids": [rep],  # IMPORTANT: exactly one ID
            "ahead_zones": _dedup_ordered(rec["ahead"]),
            "behind_zones": _dedup_ordered(rec["behind"]),
        })
    return rows


def run_step6_ci_from_step5(
    step5_df: pd.DataFrame,
    confidence: float = 0.95,
    tce_col: str = "tce_percent"
):
    """
    Step 6: Confidence interval over TCE using the already-collapsed Step 5 detail.

    Pseudocode:
      1) Take Step 5 dataframe (one row per location).
      2) Assert / enforce uniqueness by location (drop duplicates if any).
      3) Compute CI on tce_percent (no rebuild/resum of STL).
      4) Return (detail_df_used_for_CI, ci_summary_df).
    """
    detail = step5_df.copy()

    # 2) enforce uniqueness defensively (should be one row per location already)
    if "location" in detail.columns:
        if detail["location"].duplicated(keep=False).any():
            # Keep first occurrence; avoid any re-summing
            detail = detail.drop_duplicates(subset=["location"], keep="first")

    # 3) CI (dedupe_by=None because we already enforced uniqueness)
    ci_df = tce_confidence_interval_df(
        detail_df=detail,
        confidence=confidence,
        tce_col=tce_col,
        cap_abs=None,
        winsor_pct=None,
        dedupe_by=None,       # important: avoid any regrouping
        dedupe_method="mean"
    )
    return detail, ci_df

In [29]:
# 6.1, Run The Confidence Interval
    # the "detail6" dataframe provides a rows/columns view of the data
    # the "ci6" dataframe provides the statistical results of the confidence interval analysis
detail6, ci6 = run_step6_ci_from_step5(cmp_df, confidence=0.95)

In [30]:
# 6.2, View the results of the confidence interval
print(ci6)

   confidence      tce_col   n  dof   mean_tce    std_tce        se  \
0        0.95  tce_percent  10    9 -61.862402  15.751916  4.981193   

   t_critical  margin_of_error   ci_lower  ci_upper  t_statistic  \
0    2.262157        11.268242 -73.130644 -50.59416   -12.419193   

   p_value_two_sided  cohens_d  count_dropped cap_abs winsor_pct dedupe_by  \
0       5.744212e-07 -3.927294              0    None       None      None   

  dedupe_method  
0          mean  


In [31]:
# 6.3 Export to CSV 
detail6.to_csv("step_6_summary_605_d7.csv", index=False)

## Step 7, Analyst's Analysis

In [32]:
mean_tce, ci_lower, ci_upper, t_critical, t_statistic = tce_confidence_interval(
    cmp_df, confidence=0.95
)

print("Mean TCE:", mean_tce)
print("95% Confidence Interval:", (ci_lower, ci_upper))
print("t-test statistic:", t_statistic)
print("t-critical:", t_critical)

Mean TCE: -61.862401791273044
95% Confidence Interval: (-73.1306437262255, -50.59415985632059)
t-test statistic: -12.419193351662537
t-critical: 2.2621571628540993


### Mean TCE: -3.56
Traffic Count Error (TCE)
* On average across the SR99 in District 3, the Non-Traditional AADT (StreetLight) comes in about 3–4% lower than the Traditional AADT (Traffic Census).

### 95% Confidence Interval (-10.33%, 3.21%)
* We’re 95% confident the true average difference between the Non-Traditional AADT and the Traditional AADT lies between about 10% lower and about 3.2% higher.
* Because this range includes zero, we can’t claim a consistent corridor-level bias—on average, the Non-Traditional AADT may not be systematically higher or lower than the Traditional AADT data.

### T-Test Statistic (-1.06) 
* Our observed average difference (Non-Traditional AADT vs. Traditional AADT) sits about 1.06 “standard errors” below zero. The minus sign just says Non-Traditional AADT skews lower on average.

### Summary
* On SR-99 (District 3), StreetLight AADT averages about 3.6% lower than the Traffic Census.
* With 95% confidence, the true average difference could be anywhere from 10.3% lower to 3.2% higher.
* Because that range includes zero and the t-stat is −1.06 (well below the ~±2 needed at 95%), the average difference is not statistically significant.
